In [1]:
setwd(fs::path_abs("~/Local_Workspace/TesiMag"))
library(dplyr, warn.conflicts = FALSE)
library(arrow, warn.conflicts = FALSE)

source("notebooks/ds_regionali/procedure/common_steps.R")
source("src/database/query/data.R")
source("src/database/startup.R")
source("src/load/tools.R")

conns <- load_dbs()


Linking to GEOS 3.11.0, GDAL 3.5.3, PROJ 9.1.0; sf_use_s2() is TRUE



In [2]:
save_kml_orig <- function(data, key, base_path) {
    extra_meta <- dbReadTable(conns$stations, str_glue("extra_{key$dataset}"))
    data <- data |> left_join(extra_meta |> select(!c(starts_with("url"), starts_with("fk_id"))), by = "sensor_key")
    path <- file.path(base_path, key$reference_state, "original")
    if (!dir.exists(path)) {
        dir.create(path, recursive = TRUE)
    }
    data |> write.csv(file.path(path, paste0(key$dataset, ".csv")), row.names = FALSE)
    data |>
        st_md_to_sf(remove = FALSE) |>
        st_write(file.path(path, paste0(key$dataset, ".kml")), append = FALSE)
}

save_kml_merged <- function(meta, key, base_path) {
    path <- file.path(base_path, key$reference_state, "original")
    if (!dir.exists(path)) {
        dir.create(path, recursive = TRUE)
    }
    meta <- meta |> rowwise() |> mutate(across(starts_with("from_"), ~ paste0(., collapse = ";")))
    meta |> write.csv(file.path(path, "merged.csv"), row.names = FALSE)
    meta |>
        st_md_to_sf(remove = FALSE) |>
        st_write(file.path(path, "merged.kml"), append = FALSE)
}


In [7]:
regional_datasets <- tribble(
    ~reference_state, ~original_dataset, ~merged_dataset,
    # "Piemonte" , "ARPAPiemonte", "PIE",
    # "Liguria" , "ARPAL", "LIG",
    # "Lombardia" , "ARPALombardia", "LOM",
    # "Veneto" , "ARPAV", "VEN",
    # "Trentino-Alto Adige" , "TAA", "TAA2",
    "Friuli-Venezia Giulia" , "ARPAFVG", "FVG",
    # "Emilia-Romagna" , "Dext3r", "ER",
    # "Toscana" , "SIRToscana", "TOS",
    # "Umbria" , "ARPAUmbria", "UMB",
    # "Marche" , "ARPAM", "MAR",
    # "Valle D'Aosta", NA_character_, "VDA"
)


In [9]:
query_checkpoint_meta(regional_datasets$merged_dataset, "merged", conns$data) |>
    collect() |>
    left_join(regional_datasets, by = c("dataset" = "merged_dataset")) |>
    group_by(reference_state) |>
    group_walk(~ save_kml_merged(.x, .y, fs::path_abs("~/Local_Workspace/TesiMag/kml_metadata")))

Writing layer `merged' to data source 
  `/Users/davidenicoli/Local_Workspace/TesiMag/kml_metadata/Friuli-Venezia Giulia/original/merged.kml' using driver `KML'
Writing 250 features with 26 fields and geometry type Point.


In [5]:
query_checkpoint_meta(regional_datasets$original_dataset |> na.omit(), "qc1") |>
    collect() |>
    left_join(regional_datasets, by = c("dataset" = "original_dataset")) |>
    group_by(reference_state, dataset) |>
    group_walk(~ save_kml_orig(.x, .y, fs::path_abs("~/Local_Workspace/TesiMag/kml_metadata")))


Writing layer `ARPAFVG' to data source 
  `/Users/davidenicoli/Local_Workspace/TesiMag/kml_metadata/Friuli-Venezia Giulia/original/ARPAFVG.kml' using driver `KML'
Writing 53 features with 27 fields and geometry type Point.


In [6]:
query_checkpoint_meta(c("SCIA", "ISAC"), "raw") |>
    collect() |>
    mutate(reference_state = coalesce(state, "Unknown")) |>
    group_by(reference_state, dataset) |>
    group_walk(~ save_kml_orig(.x, .y, fs::path_abs("~/Local_Workspace/TesiMag/kml_metadata")))

Writing layer `ISAC' to data source 
  `/Users/davidenicoli/Local_Workspace/TesiMag/kml_metadata/Abruzzo/original/ISAC.kml' using driver `KML'
Writing 186 features with 32 fields and geometry type Point.
Writing layer `SCIA' to data source 
  `/Users/davidenicoli/Local_Workspace/TesiMag/kml_metadata/Abruzzo/original/SCIA.kml' using driver `KML'
Writing 151 features with 28 fields and geometry type Point.
Writing layer `ISAC' to data source 
  `/Users/davidenicoli/Local_Workspace/TesiMag/kml_metadata/Basilicata/original/ISAC.kml' using driver `KML'
Writing 58 features with 32 fields and geometry type Point.
Writing layer `SCIA' to data source 
  `/Users/davidenicoli/Local_Workspace/TesiMag/kml_metadata/Basilicata/original/SCIA.kml' using driver `KML'
Writing 62 features with 28 fields and geometry type Point.
Writing layer `ISAC' to data source 
  `/Users/davidenicoli/Local_Workspace/TesiMag/kml_metadata/Calabria/original/ISAC.kml' using driver `KML'
Writing 114 features with 32 fields 

In [33]:
open_dataset(archive_path(c("SCIA", "ISAC"), "metadata", "qc1")) |>
    collect() |>
    filter(is.na(state))

sensor_key,sensor_id,station_key,sensor_first,sensor_last,station_id,name,network,town,province_full,...,user_code,series_key,kind,lon,lat,elevation,series_id,dataset,series_first,series_last
<int>,<chr>,<int>,<date>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,...,<chr>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<date>,<date>
138,NA,2377,2010-04-01,2022-12-31,NA,Ancona,Mareografica,NA,NA,...,70015,2377,unknown,13.50600,43.62400,0,6055,SCIA,2010-04-01,2022-12-31
285,NA,2681,2005-07-27,2022-01-17,NA,BOUSSON,Sinottica,NA,NA,...,160580,2681,unknown,6.75000,44.93300,1400,6706,SCIA,2005-07-27,2022-01-17
338,NA,2367,2009-09-09,2022-12-31,NA,Bari,Mareografica,NA,NA,...,70005,2367,unknown,16.86600,41.13900,0,6045,SCIA,2009-09-09,2022-12-31
479,NA,2799,2005-07-27,2016-07-14,NA,CAPE CIRCEO,Sinottica,NA,NA,...,162490,2799,unknown,13.05000,41.21700,3,6825,SCIA,2005-07-27,2016-07-14
480,NA,2869,2005-07-27,2022-04-28,NA,CAPE SPARTIVENTO,Sinottica,NA,NA,...,164690,2869,unknown,16.06700,37.91700,17,6896,SCIA,2005-07-27,2022-04-28
482,NA,2895,1951-01-01,2022-12-31,NA,CAPO BELLAVISTA,Sinottica,NA,NA,...,165500,2895,unknown,9.71700,39.93300,150,6922,SCIA,1951-01-01,2022-12-31
483,NA,2880,1975-05-11,2022-12-31,NA,CAPO CACCIA,Sinottica,NA,NA,...,165220,2880,unknown,8.16700,40.56700,204,6907,SCIA,1975-05-11,2022-12-31
484,NA,2881,2005-07-27,2005-11-16,NA,CAPO CACCIA,Sinottica,NA,NA,...,165230,2881,unknown,8.16700,40.56700,203,6908,SCIA,2005-07-27,2005-11-16
488,NA,2746,1963-11-30,2022-12-31,NA,CAPO MELE,Sinottica,NA,NA,...,161530,2746,unknown,8.16700,43.95000,221,6771,SCIA,1963-11-30,2022-12-31
